In [1]:
import pandas as pd
import glob
import os
import shutil
import re
from joblib import Parallel, delayed
import nibabel as nib
import numpy as np
import random
import seaborn as sns
from datetime import datetime

In [2]:
import sys
sys.path.append('/media/raid/orco/data/ICH/code/neurorad_bids_utils')
from bids_utils.utils import utils, mask_utils

In [3]:
dataset_names = ['Dataset011_5FoldCV', 'Dataset012_5FoldCV', 'Dataset013_5FoldCV', 'Dataset014_5FoldCV', 'Dataset015_5FoldCV']

In [4]:
preds = glob.glob('/home/orco/data/VentrikelCNN/derivatives/nnUNet/nnUNet_raw/Dataset01*/test*/*.nii.gz')
len(preds)

154

In [5]:
df = pd.DataFrame(preds)
df.columns = ['prediction']
df['gt'] = df.prediction.apply(lambda x: x.replace('test_fold1', 'labelsTs'))
df['ds'] = df.prediction.apply(lambda x: x.split('/')[-3])

In [6]:
res = []
for i,j in zip(list(df.prediction),list(df['gt'])):
    res.append([i,j] + utils.mp_dice_n(i, j, [1,2,3,4,5]))

/media/raid/orco/data/ICH/code/neurorad_bids_utils/bids_utils/utils/utils.py:24: RuntimeWarning: invalid value encountered in scalar divide
  return 2*intersection/(a_arr.sum()+b_arr.sum())


In [7]:
df = pd.DataFrame(res)

In [8]:
df.head()

,0,1,2,3,4,5,6
0,/home/orco/data/VentrikelCNN/derivatives/nnUNe...,/home/orco/data/VentrikelCNN/derivatives/nnUNe...,0.945415,0.955518,0.969490,0.963869,0.845411
1,/home/orco/data/VentrikelCNN/derivatives/nnUNe...,/home/orco/data/VentrikelCNN/derivatives/nnUNe...,0.981807,0.984666,0.965899,0.948001,0.816176
2,/home/orco/data/VentrikelCNN/derivatives/nnUNe...,/home/orco/data/VentrikelCNN/derivatives/nnUNe...,0.979531,0.961349,0.955765,0.766721,0.392157
3,/home/orco/data/VentrikelCNN/derivatives/nnUNe...,/home/orco/data/VentrikelCNN/derivatives/nnUNe...,0.944871,0.944085,0.834331,0.920129,0.795181
4,/home/orco/data/VentrikelCNN/derivatives/nnUNe...,/home/orco/data/VentrikelCNN/derivatives/nnUNe...,0.891487,0.974966,0.931481,0.926316,0.000000


In [9]:
df.columns = ['prediction', 'gt', 'Vol_left', 'Vol_right', 'Vol_3rd', 'Vol_4th', 'Vol_Acq']


In [10]:
df['sid'] = df.prediction.apply(lambda x: os.path.basename(x).split('_')[1].replace('.nii.gz',''))

In [14]:
df['sid'] = 'sub-V' + df.sid

In [15]:
df.head()

,prediction,gt,Vol_left,Vol_right,Vol_3rd,Vol_4th,Vol_Acq,sid
0,/home/orco/data/VentrikelCNN/derivatives/nnUNe...,/home/orco/data/VentrikelCNN/derivatives/nnUNe...,0.945415,0.955518,0.969490,0.963869,0.845411,sub-V083
1,/home/orco/data/VentrikelCNN/derivatives/nnUNe...,/home/orco/data/VentrikelCNN/derivatives/nnUNe...,0.981807,0.984666,0.965899,0.948001,0.816176,sub-V108
2,/home/orco/data/VentrikelCNN/derivatives/nnUNe...,/home/orco/data/VentrikelCNN/derivatives/nnUNe...,0.979531,0.961349,0.955765,0.766721,0.392157,sub-V115
3,/home/orco/data/VentrikelCNN/derivatives/nnUNe...,/home/orco/data/VentrikelCNN/derivatives/nnUNe...,0.944871,0.944085,0.834331,0.920129,0.795181,sub-V074
4,/home/orco/data/VentrikelCNN/derivatives/nnUNe...,/home/orco/data/VentrikelCNN/derivatives/nnUNe...,0.891487,0.974966,0.931481,0.926316,0.000000,sub-V266


In [20]:
frees_etivs = '/home/orco/data/VentrikelCNN/code/stats/freesurfer_eTIVs.csv'
etivs = pd.read_csv(frees_etivs)

In [19]:
etivs.head()

,path,eTIV
0,/data/gpfs-1/users/dellorca_c/work/VentrikelCN...,468381
1,/data/gpfs-1/users/dellorca_c/work/VentrikelCN...,1399601
2,/data/gpfs-1/users/dellorca_c/work/VentrikelCN...,474830
3,/data/gpfs-1/users/dellorca_c/work/VentrikelCN...,959807
4,/data/gpfs-1/users/dellorca_c/work/VentrikelCN...,793015


In [24]:
etivs['sid'] = etivs.path.apply(lambda x: x.split('/')[9])
etivs.head()                   

,path,eTIV,sid
0,/data/gpfs-1/users/dellorca_c/work/VentrikelCN...,468381,sub-V001
1,/data/gpfs-1/users/dellorca_c/work/VentrikelCN...,1399601,sub-V002
2,/data/gpfs-1/users/dellorca_c/work/VentrikelCN...,474830,sub-V003
3,/data/gpfs-1/users/dellorca_c/work/VentrikelCN...,959807,sub-V004
4,/data/gpfs-1/users/dellorca_c/work/VentrikelCN...,793015,sub-V005


In [28]:
df = df.merge(etivs, on='sid').drop(columns='path')
df = df[['sid', 'eTIV'] +[i for i in df.columns if i.startswith('Vol')]]
df.head()

,sid,eTIV,Vol_left,Vol_right,Vol_3rd,Vol_4th,Vol_Acq
0,sub-V083,1254638,0.945415,0.955518,0.969490,0.963869,0.845411
1,sub-V108,408478,0.981807,0.984666,0.965899,0.948001,0.816176
2,sub-V115,1218545,0.979531,0.961349,0.955765,0.766721,0.392157
3,sub-V074,1300745,0.944871,0.944085,0.834331,0.920129,0.795181
4,sub-V266,1646525,0.891487,0.974966,0.931481,0.926316,0.000000


In [29]:
date = '{}{}{}'.format(datetime.now().year, datetime.now().month, datetime.now().day)
date

'2024319'

In [30]:
df.to_csv('./data/volumes_nnUNet_5classes_{}_{}.csv'.format('5FoldCV', date), index=False)